# How to create features from textual data? Bag of Words..?

1. Each document is made of bag of words.
2. Ignore Sentence structure.
3. Care about frequency of words across corpus.

The BoW model learns a vocabulary from all of the documents, then models each document by counting the number of times each word appears. For example, consider the following two sentences:

Sentence 1: "The cat sat on the hat"<br>
Sentence 2: "The dog ate the cat and the hat"

From these two sentences, our vocabulary is as follows:
{ the, cat, sat, on, hat, dog, ate, and } of size 8

Hence, length of our feature vector would be of size 8

FeatureVector 1: { 2, 1, 1, 1, 1, 0, 0, 0 }<br>
FeatureVector 2: { 3, 1, 0, 0, 1, 1, 1, 1}

# ML Model for Sentiment classification

Steps To Follow for creating a Machine Learning model for Sentiment Classification
1. Load the training data using pandas
2. Perform text pre-processing
3. Tokenize the text into words
4. Create BoW feature vector of each document
5. Use the feature vectors to build an ML model for classification


In [1]:
import pandas as pd
data_dir = 'data/'

In [2]:
df_train = pd.read_csv(data_dir + 'labeledTrainData.tsv', sep="\t")

In [3]:
df_train.columns

Index(['id', 'sentiment', 'review'], dtype='object')

In [7]:
#Take only the columns you need

df_train=df_train[['sentiment', 'review']]

In [8]:
df_train.columns

Index(['sentiment', 'review'], dtype='object')

In [12]:
#Data Cleaning and Text Preprocessing

from bs4 import BeautifulSoup
import re

def preproc(line):
    line=line.lower()
    text = BeautifulSoup(line, "lxml").get_text()
    return re.sub("[\"\\\\]","", text)

In [13]:
df_train['review']=df_train['review'].apply(preproc)

In [14]:
df_train['review'][0]

"with all this stuff going down at the moment with mj i've started listening to his music, watching the odd documentary here and there, watched the wiz and watched moonwalker again. maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. some of it has subtle messages about mj's feeling towards the press and also the obvious message of drugs are bad m'kay.visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring. some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him.the actual feature film bit when it finally starts is only on for 20 mi

In [16]:
#Tokenization

import spacy
nlp = spacy.load('en')

In [18]:
%time tokenized_docs=[[word.text for word in nlp(doc)] for doc in df_train['review']]

# Stopwords

Common occuring words like (a, an, the etc) which are usually removed while performing language processing.
Intention is to have more focus on important words which provides meaning to the text.

In [21]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [36]:
len(stopwords)
list(stopwords)[0:20]

['ours',
 'what',
 'bottom',
 'put',
 'latterly',
 'each',
 'whether',
 'whence',
 'along',
 'take',
 'do',
 'made',
 'were',
 'except',
 'give',
 'once',
 'therefore',
 'become',
 'thereupon',
 'unless']

In [40]:
def remove_stop_words(words):
    imp_words=[w for w in words if not w in stopwords]
    return " ".join(imp_words)

In [41]:
 meaning_words=list(map(remove_stop_words, tokenized_docs))

# Create Bag of Words features

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
vectorizer = CountVectorizer(stop_words = None, max_features = 5000) 

In [42]:
train_data_features = vectorizer.fit_transform(meaning_words)

In [43]:
type(train_data_features)

scipy.sparse.csr.csr_matrix

In [44]:
train_data_features = train_data_features.toarray()

In [45]:
type(train_data_features)

numpy.ndarray

In [46]:
train_data_features.shape

(25000, 5000)

In [57]:
#Take a look at the words in vocab
vocab = vectorizer.get_feature_names()

In [60]:
#vocab[0:20]
vocab[100:120]

['account',
 'accuracy',
 'accurate',
 'accused',
 'achieve',
 'achieved',
 'achievement',
 'acid',
 'act',
 'acted',
 'acting',
 'action',
 'actions',
 'activities',
 'actor',
 'actors',
 'actress',
 'actresses',
 'acts',
 'actual']

In [67]:
# Print Count of Vocab words

import numpy as np
dist = np.sum(train_data_features, axis=0)
word_freq=[(tag, count) for tag, count in zip(vocab, dist)]

In [69]:
word_freq[100:120]

[('account', 186),
 ('accuracy', 81),
 ('accurate', 284),
 ('accused', 123),
 ('achieve', 179),
 ('achieved', 139),
 ('achievement', 124),
 ('acid', 90),
 ('act', 1251),
 ('acted', 658),
 ('acting', 6490),
 ('action', 3353),
 ('actions', 311),
 ('activities', 83),
 ('actor', 2389),
 ('actors', 4485),
 ('actress', 1219),
 ('actresses', 368),
 ('acts', 394),
 ('actual', 793)]

# Split into training and test set

In [49]:
n=20000

X_train, X_test=train_data_features[:n], train_data_features[n:]

In [50]:
(X_train.shape, X_test.shape)

((20000, 5000), (5000, 5000))

In [52]:
target=df_train['sentiment']

In [55]:
y_train, y_test=target[0:n], target[n:target.shape[0]]

In [56]:
(y_train.shape, y_test.shape)

((20000,), (5000,))

In [61]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
forest = RandomForestClassifier(n_estimators = 20) 

In [63]:
%time model=forest.fit(X_train, y_train)

In [64]:
model.score(X_train,y_train)

0.9991

In [65]:
model.score(X_test,y_test)

0.8056